In [ ]:
%%capture
pip install numpy pandas

In [ ]:
import numpy as np
import pandas as pd

### **1. Identify people who are in cohort_users after completing form_entry**

In [ ]:
form_entry = pd.read_csv('../sources/modified_form_entry.csv', low_memory=False)

In [ ]:
form_entry.columns

In [ ]:
cohort_users = pd.read_csv('../sources/cohort_users.csv')

In [ ]:
cohort_users

In [ ]:
auth = pd.read_csv('../sources/auth_user.csv')

In [ ]:
# Merge cohort_users and auth
cohort_users_auth = cohort_users.merge(auth, left_on='user_id', right_on='id', how='left')

In [ ]:
cohort_users_auth.columns

In [ ]:
cohort_users_auth = cohort_users_auth[['email', 'role', 'created_at', 'cohort_id']]

In [ ]:
# Consider only users with student as role
cohort_users_auth = cohort_users_auth[cohort_users_auth['role'].str.lower()=='student']

In [ ]:
# Delete "+00:00" from dates
def correct_date(time):
    if pd.isna(time):
         return time
    else:
        sep = '+'
        stripped = time.split(sep, 1)[0]
        return stripped

In [ ]:
cohort_users_auth['created_at'] = cohort_users_auth['created_at'].apply(correct_date)

In [ ]:
form_entry['created_at'] = form_entry['created_at'].apply(correct_date)

In [ ]:
cohort_users_auth['datetime_cohort_user_creation'] = pd.to_datetime(cohort_users_auth['created_at'])

In [ ]:
form_entry['datetime_form_entry_creation'] = pd.to_datetime(form_entry['created_at'])

In [ ]:
cohort_users_auth

In [ ]:
form_entry = pd.merge(form_entry, cohort_users_auth, on='email', how='left')

In [ ]:
form_entry = form_entry.rename(columns={'created_at_x':'form_created_at', 'created_at_y':'cohort_user_created_at'})

In [ ]:
def identify_cohort_users(row):
    cohort_user_creation = row['datetime_cohort_user_creation']
    form_entry_creation = row['datetime_form_entry_creation']

    if (pd.notnull(cohort_user_creation)) and (cohort_user_creation > form_entry_creation):
        return 1
    else:
        return 0

In [ ]:
form_entry['in_cohort_user'] = form_entry.apply(identify_cohort_users, axis=1)

In [ ]:
form_entry = form_entry.drop_duplicates(subset=['email', 'course', 'language'], keep='last')

In [ ]:
form_entry.columns

### **2. Clean some form_entry columns**

In [ ]:
form_entry['course'] = form_entry['course'].replace(['full-stack-ft', 'full_stack', 'full-stack', 'software-engineering',
                                         'coding-introduction', 'outcomes', 'Full Stack'], 'full-stack')

In [ ]:
form_entry['course'] = form_entry['course'].replace(['machine-learning', 'machine-learning-enginnering', 'datascience-ml', 'datascience'], 
                                        'machine-learning-engineering')

In [ ]:
"""form_entry['academy_id'] = np.where((form_entry['location'] == 'maracaibo-venezuela'),
                                2.0, form_entry['academy_id'])
form_entry['academy_id'] = np.where((form_entry['location'] == 'toronto-canada'),
                                4.0, form_entry['academy_id'])
form_entry['academy_id'] = np.where((form_entry['location'] == 'costa-rica') |
                                (form_entry['location'] == 'sanjose-uruguay'),
                                7.0, form_entry['academy_id'])"""

In [ ]:
form_entry['fullname'] = form_entry['first_name'].fillna('') + str(' ') + form_entry['last_name'].fillna('')

In [ ]:
# form_entry['location'] = form_entry['location'].replace(['los-cortijos-caracas'], 'caracas-venezuela')

In [ ]:
# form_entry['location'] = form_entry['location'].replace(['lisboa-portugal'], 'lisbon-portugal')

In [ ]:
form_entry['language'] = form_entry['language'].replace('us', 'en')

In [ ]:
# Delete "+00:00"
def correct_date(time):
    if pd.isna(time):
         return time
    else:
        sep = '+'
        stripped = time.split(sep, 1)[0]
        return stripped

In [ ]:
form_entry.columns

In [ ]:
form_entry['created_at'] = form_entry['created_at'].apply(correct_date)

In [ ]:
form_entry['updated_at'] = form_entry['updated_at'].apply(correct_date)

In [ ]:
form_entry['ac_expected_cohort_date'] = form_entry['ac_expected_cohort_date'].apply(correct_date)

### **3. Identify upcoming cohorts and select only needed columns**

In [ ]:
cohorts = pd.read_csv('../sources/cohorts.csv')

In [ ]:
cohorts.columns

In [ ]:
cohorts['academy_id'].info()

In [ ]:
# Delete "+00:00"
def correct_date(time):
    if pd.isna(time):
         return time
    else:
        sep = '+'
        stripped = time.split(sep, 1)[0]
        return stripped

In [ ]:
cohorts['kickoff_date'] = cohorts['kickoff_date'].apply(correct_date)

In [ ]:
cohorts['datetime_kickoff'] = pd.to_datetime(cohorts['kickoff_date'])

In [ ]:
current_time = pd.Timestamp.now()

In [ ]:
upcoming_cohorts = cohorts[cohorts['datetime_kickoff']>current_time]

In [ ]:
upcoming_cohorts = upcoming_cohorts[upcoming_cohorts['never_ends']==False]

In [ ]:
upcoming_cohorts = upcoming_cohorts[(upcoming_cohorts['stage']!='DELETED')]

In [ ]:
upcoming_cohorts = upcoming_cohorts[['slug', 'academy_id', 'datetime_kickoff']]

In [ ]:
upcoming_cohorts

### **4. Complete ac_expected_cohort and add cohort_assignation_error**

In [ ]:
form_entry['datetime_expected_cohort_date'] = pd.to_datetime(form_entry['ac_expected_cohort_date'])

In [ ]:
def assign_expected_cohort(row, df=upcoming_cohorts):
    
    datetime_expected = row['datetime_expected_cohort_date']
    expected_date = row['ac_expected_cohort_date']
    ac_expected_cohort = row['ac_expected_cohort']
    fe_academy_id = row['academy_id']
    fe_course = row['course']
    in_cohort_user = row['in_cohort_user']

    if (pd.notnull(ac_expected_cohort)) or (in_cohort_user==1):
        return ac_expected_cohort
    elif pd.isna(expected_date):
        return np.nan
    elif pd.isna(fe_academy_id):
        return 'Missing academy'
    elif pd.isna(fe_course):
        return 'Missing course'
    else:

        df_filtered = df[df['academy_id']==int(fe_academy_id)]

        if fe_course == 'machine-learning-engineering':
            df_filtered = df_filtered[df_filtered['slug'].str.contains('-ml-')]
        else:
            df_filtered = df_filtered[~df_filtered['slug'].str.contains('-ml-')]

        if df_filtered.empty:
            return 'Missing upcoming cohort'
        else:
            datetime_expected = datetime_expected.replace(tzinfo=None)
            df_filtered = df_filtered[df_filtered['datetime_kickoff'] >= datetime_expected]
            df_filtered['time_diff'] = abs(df_filtered['datetime_kickoff'] - datetime_expected)    
            min_idx = df_filtered['time_diff'].idxmin()
            expected_cohort = df_filtered.loc[min_idx, 'slug']
            return expected_cohort

In [ ]:
form_entry['date_assigned_cohort'] = form_entry.apply(assign_expected_cohort, axis=1)

In [ ]:
error_values = ['Missing academy', 'Missing course', 'Missing upcoming cohort']

In [ ]:
form_entry.loc[form_entry['date_assigned_cohort'].isin(error_values), 'cohort_assignation_error'] = form_entry['date_assigned_cohort']

In [ ]:
form_entry.loc[~form_entry['date_assigned_cohort'].isin(error_values), 'ac_expected_cohort'] = form_entry['date_assigned_cohort']

In [ ]:
form_entry.loc[form_entry['date_assigned_cohort'].isna(), ['ac_expected_cohort', 'cohort_assignation_error']] = np.NaN

In [ ]:
form_entry = form_entry.drop(columns=['date_assigned_cohort'])

### **5. Look for inconsistencies between academy and location**

In [ ]:
academies_alias = pd.read_csv('../sources/academies_alias.csv')

In [ ]:
dict_locations = academies_alias.groupby('name')['slug'].apply(list).to_dict()

In [ ]:
dict_locations

In [ ]:
academies_alias = academies_alias.drop(columns=['slug', 'status'])

In [ ]:
academies_alias = academies_alias.rename(columns={'name':'campus'})

In [ ]:
academies_alias = academies_alias.drop_duplicates()

In [ ]:
academies_alias

In [ ]:
form_entry = form_entry.merge(academies_alias, on='academy_id', how='left')

In [ ]:
def campus_check(row):
    
    campus = row['campus']
    location = row['location']
    
    if pd.isna(campus):
        return 'No academy assigned'
    else:
        if campus in dict_locations:
            if location in dict_locations[campus]:
                return 'Location matches academy'
            else:
                return 'Location does not match academy'
        else:
            return 'Unknown academy'

In [ ]:
form_entry['campus_check'] = form_entry.apply(campus_check, axis=1)

In [ ]:
form_entry.columns

In [ ]:
form_entry.to_csv('../output/form_entry_processed.csv', index=False)

In [ ]:
form_entry